<a href="https://colab.research.google.com/github/Nedac23/Machine-Learning/blob/main/MLHW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Question 1:

In [ ]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, confusion_matrix

In [ ]:
# Function to compute mean and variance, 5 points
def mean_variance(X):
  #gets the mean and vairence for each pixel
    mean = np.mean(X, axis=0)
    variance = np.var(X, axis=0)
    return mean, variance

In [ ]:
#NOT USED
#def gaussian_pdf(x, mean, var):
   # l = (1 / (np.sqrt(var) * np.sqrt(2 * np.pi))) * np.exp(-((x - mean) ** 2) / (2 * var))
    #return l

In [ ]:
# Function to compute Gaussian probability density, 5 points
#uses guassina pdf but instead of how its used in class logs are used to prevent underflwo error
#when using the above one I had a very low accucary of around 9% which didnt make sense untill i discovered underflow error
def gaussian_log_pdf(x, mean, var):
    var = np.maximum(var, 1e-6)  # Ensure variance is never zero
    return -0.5 * (np.log(2 * np.pi * var) + ((x - mean) ** 2) / var)

In [ ]:
# Function to compute priors, 5 points
def compute_priors(y):
    classes, counts = np.unique(y, return_counts=True)
    priors = counts / len(y)
    return priors

In [ ]:
# # Prediction, 10 points
# def predict(X, Y, means, variances, priors):

#     # m = {}
#     # j = 0
#     predictions = []
#     for x in X:
#       # pred = {}
#       # n = {}
#       # dist = 1
#       class_likelihoods = []
#       for i in range(10):
#         likelihood = priors[i]
#         # for p in range(len(x)):
#         #   dist = gaussian_pdf(x,means[i],variances[i]) * dist
#         #   like = dist * priors[i]
#         #   pred[p] = like
#         # n[i] = np.argmax(list(pred.values()))
#         # pred.clear()

#         for p in range(len(x)):  # For each pixel
#           likelihood *= gaussian_pdf(x[p], means[i][p], variances[i][p])
#         class_likelihoods.append(likelihood)
#       predicted_class = np.argmax(class_likelihoods)
#       predictions.append(predicted_class)




#       # final = np.argmax(list(n.values()))
#       # m[j] = final
#       # pred.clear()
#       # j = j + 1

#     #return np.array(list(m.values()))
#     return np.array(predictions)

In [ ]:
# Prediction, 10 points
def predict_log(X, means, variances, priors):
    #ChatGpt was used to discover an issue with underflow values causing a low accuacy.
    #This code addresses this by using logarithm calculations rather than the above funciton.
    predictions = []

    for x in X:  # For each image
        class_log_likelihoods = []

        for i in range(10):  # For each class
            log_likelihood = np.log(priors[i])
            for p in range(len(x)):  # For each pixel
                log_likelihood += gaussian_log_pdf(x[p], means[i][p], variances[i][p])
            class_log_likelihoods.append(log_likelihood)

        predicted_class = np.argmax(class_log_likelihoods)
        predictions.append(predicted_class)

    return np.array(predictions)

In [ ]:
# Load the dataset
digits = load_digits()
X, y = digits.data, digits.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#print(len(y_train))
# Normalize the feature data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Training: Compute mean, variance, and priors for each class
means = {}
variances = {}
priors = compute_priors(y_train)
#priors is  an ordered array of the prior prob of each class
print(len(X_train_scaled))
print(len(y_train))
for c in np.unique(y_train):
  X_c = X_train_scaled[y_train == c]
  #print(c)
  print(len(X_c))
  #c is the number it is labeled as so for ex. 0 - 9
  # X_c is the data for that particluar number
  #print(X_c)
  means[c], variances[c] = mean_variance(X_c)
  print(f"Class {c} Mean Shape: {means[c].shape}, Variance Shape: {variances[c].shape}")


1437
1437
145
Class 0 Mean Shape: (64,), Variance Shape: (64,)
154
Class 1 Mean Shape: (64,), Variance Shape: (64,)
144
Class 2 Mean Shape: (64,), Variance Shape: (64,)
149
Class 3 Mean Shape: (64,), Variance Shape: (64,)
135
Class 4 Mean Shape: (64,), Variance Shape: (64,)
135
Class 5 Mean Shape: (64,), Variance Shape: (64,)
146
Class 6 Mean Shape: (64,), Variance Shape: (64,)
145
Class 7 Mean Shape: (64,), Variance Shape: (64,)
144
Class 8 Mean Shape: (64,), Variance Shape: (64,)
140
Class 9 Mean Shape: (64,), Variance Shape: (64,)


In [ ]:
# Evaluate the model on scaled data
y_pred_scaled = predict_log(X_test_scaled, means,variances,priors)
print(len(X_test_scaled))
print(len(y_pred_scaled))
print(len(y_test))

accuracy_scaled = np.mean(y_pred_scaled == y_test)
# Evaluation
print(f"Accuracy after scaling: {accuracy_scaled:.4f}")
conf_matrix = confusion_matrix(y_test, y_pred_scaled)
f1_micro = f1_score(y_test, y_pred_scaled, average='micro')
f1_macro = f1_score(y_test, y_pred_scaled, average='macro')
f1_weighted = f1_score(y_test, y_pred_scaled, average='weighted')

precision_micro = precision_score(y_test, y_pred_scaled, average='micro')
precision_macro = precision_score(y_test, y_pred_scaled, average='macro')
precision_weighted = precision_score(y_test, y_pred_scaled, average='weighted')

recall_micro = recall_score(y_test, y_pred_scaled, average='micro')
recall_macro = recall_score(y_test, y_pred_scaled, average='macro')
recall_weighted = recall_score(y_test, y_pred_scaled, average='weighted')

360
360
360
Accuracy after scaling: 0.8528


In [ ]:
# Display the results
print(f"Confusion Matrix:\n{conf_matrix}")

print("\nF1 Scores:")
print(f"Micro: {f1_micro:.4f}")
print(f"Macro: {f1_macro:.4f}")
print(f"Weighted: {f1_weighted:.4f}")

print("\nPrecision Scores:")
print(f"Micro: {precision_micro:.4f}")
print(f"Macro: {precision_macro:.4f}")
print(f"Weighted: {precision_weighted:.4f}")

print("\nRecall Scores:")
print(f"Micro: {recall_micro:.4f}")
print(f"Macro: {recall_macro:.4f}")
print(f"Weighted: {recall_weighted:.4f}")

Confusion Matrix:
[[31  0  0  0  0  1  0  1  0  0]
 [ 0 22  0  0  0  0  0  0  4  2]
 [ 0  2 20  1  0  0  1  0  9  0]
 [ 0  0  1 30  0  1  0  0  2  0]
 [ 0  0  0  0 37  0  2  7  0  0]
 [ 0  0  0  1  0 44  1  1  0  0]
 [ 0  0  0  0  0  0 35  0  0  0]
 [ 0  0  0  0  0  1  0 33  0  0]
 [ 0  1  0  0  0  0  0  2 27  0]
 [ 0  1  1  1  0  2  0  3  4 28]]

F1 Scores:
Micro: 0.8528
Macro: 0.8486
Weighted: 0.8540

Precision Scores:
Micro: 0.8528
Macro: 0.8682
Weighted: 0.8779

Recall Scores:
Micro: 0.8528
Macro: 0.8525
Weighted: 0.8528


Question 2:

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from scipy.stats import norm

In [ ]:
class MixedNB:
  #chatgpt was used to help flesh out the structure of the class
  #it was used to help understand and find the best implementation method for parts of the fit and predict functions.
    def __init__(self):
      self.class_probs = None
      self.continuous = {}
      self.categorical = {}


    def fit(self, X, y, continuous_features):
     #priors
      classes, counts = np.unique(y, return_counts=True)
      priors = counts / len(y)
      self.class_probs = priors

      #intializing dictionaries to store mean and var and propabilites for categories
      #intializes number of samels and features based on X the data
      samples, features = X.shape
      self.continuous = {}
      self.categorical = {}

      #means and var for contuinous to be used in guassina
      for con in continuous_features:
          self.continuous[con] = {}
          for c in np.unique(y):
            #gets value for that calss and feature
              vals = X[y == c, con]
              #sets mean var for each to be used later in guassian
              self.continuous[con][c] = {
                  'mean': np.mean(vals),
                  'variance': np.var(vals)
              }

      #finds categorical features from the X and conti lists
      categorical_features = [i for i in range(features) if i not in continuous_features]
      #categorical_features = np.setdiff1d(X,continuous_features, assume_unique=False)
      for cat in categorical_features:
          self.categorical[cat] = {}
          for c in np.unique(y):
            #gets value for that calss and feature
              vals = X[y == c, cat]
              #calcules probaboilities in the same was as proirs for each val
              unique_values, counts = np.unique(vals, return_counts=True)
              probs = counts / counts.sum()
              #adds to probs for categories
              self.categorical[cat][c] = (unique_values, probs)



    def predict(self, X):
      samples = X.shape[0]
      predictions = []

      for i in range(samples):
        #for each sample
          sample = X[i]
          class_log_probs = []

          for cIdx, cProb in enumerate(self.class_probs):
            #for each class and probability
            #using logs to account for underflow
              log_prob = np.log(cProb)

              for feature, value in enumerate(sample):
                  #for each feature and value
                  #if contious
                  if feature in self.continuous:
                      if cIdx in self.continuous[feature]:
                        #pulls the mean and vairence for the feature in the class
                          params = self.continuous[feature][cIdx]
                          mean = params['mean']
                          var = params['variance']
                          if var <= 0:
                            var = 1e-6

                          #caluating guassinan
                          log_likelihood = -0.5 * np.log(2 * np.pi * var) - ((value - mean) ** 2) / (2 * var)
                          log_prob += log_likelihood

                  # category
                  else:
                      if cIdx in self.categorical[feature]:
                          categories, probs = self.categorical[feature][cIdx]
                          #pulls the probability and category for the feature in the class
                          if value in categories:
                            matching_index = np.where(categories == value)[0][0]
                            prob = probs[matching_index]
                          else:
                            #prob is small
                              prob = 1e-10
                          log_prob += np.log(prob)
              #adds probabilities
              class_log_probs.append(log_prob)

          #predicts best class
          predictions.append(np.argmax(class_log_probs))

      return np.array(predictions)




In [ ]:
# Load the dataset
adult = fetch_openml(name='adult', version=2)
X, y = adult.data, adult.target

# Convert to DataFrame for easier preprocessing
X_df = pd.DataFrame(X, columns=adult.feature_names)
y_encoded = LabelEncoder().fit_transform(y)

# Impute missing values
imputer = SimpleImputer(strategy='most_frequent')
X_df_imputed = pd.DataFrame(imputer.fit_transform(X_df), columns=X_df.columns)

# Encode categorical variables
X_df_encoded = X_df_imputed.apply(lambda col: LabelEncoder().fit_transform(col.astype(str)) if col.dtype.name == 'category' or col.dtype == object else col)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X_df_encoded, y_encoded, test_size=0.2, random_state=42)

# Identify continuous features
continuous_features = [i for i, col in enumerate(X_df.columns) if X_df[col].dtype.name != 'category' and X_df[col].dtype != object]

In [ ]:
# Instantiate and train the model
model = MixedNB()
model.fit(X_train.values, y_train, continuous_features=continuous_features)

In [ ]:
# Predict on the test set
y_pred = model.predict(X_test.values)

# Evaluate the model
accuracy = np.mean(y_pred == y_test)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.8294


In [ ]:
# Calculations
conf_matrix = confusion_matrix(y_test, y_pred)
f1_micro = f1_score(y_test, y_pred, average='micro')
f1_macro = f1_score(y_test, y_pred, average='macro')
f1_weighted = f1_score(y_test, y_pred, average='weighted')

precision_micro = precision_score(y_test, y_pred, average='micro')
precision_macro = precision_score(y_test, y_pred, average='macro')
precision_weighted = precision_score(y_test, y_pred, average='weighted')

recall_micro = recall_score(y_test, y_pred, average='micro')
recall_macro = recall_score(y_test, y_pred, average='macro')
recall_weighted = recall_score(y_test, y_pred, average='weighted')

# Display the results
print(f"Confusion Matrix:\n{conf_matrix}")

print("\nF1 Scores:")
print(f"Micro: {f1_micro:.4f}")
print(f"Macro: {f1_macro:.4f}")
print(f"Weighted: {f1_weighted:.4f}")

print("\nPrecision Scores:")
print(f"Micro: {precision_micro:.4f}")
print(f"Macro: {precision_macro:.4f}")
print(f"Weighted: {precision_weighted:.4f}")

print("\nRecall Scores:")
print(f"Micro: {recall_micro:.4f}")
print(f"Macro: {recall_macro:.4f}")
print(f"Weighted: {recall_weighted:.4f}")

Confusion Matrix:
[[6676  803]
 [ 864 1426]]

F1 Scores:
Micro: 0.8294
Macro: 0.7601
Weighted: 0.8286

Precision Scores:
Micro: 0.8294
Macro: 0.7626
Weighted: 0.8278

Recall Scores:
Micro: 0.8294
Macro: 0.7577
Weighted: 0.8294


References and AI Usage:

http://numpy.org/doc/2.2/index.html

The author acknowledges the use of ChatGPT in the preparation of this assignment to brainstorm and understand approaches to the problems and better understand the content. ChatGPT was used to help understand and implement parts of the Naive Bayes classifier.